In [84]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bajlecgrausayegh/keras/default/1/config.json
/kaggle/input/bajlecgrausayegh/keras/default/1/metadata.json
/kaggle/input/bajlecgrausayegh/keras/default/1/model.weights.h5
/kaggle/input/dl-itba-cifar-100-2024-q-1/y_train_fine.npy
/kaggle/input/dl-itba-cifar-100-2024-q-1/y_train_coarse.npy
/kaggle/input/dl-itba-cifar-100-2024-q-1/fine_label_names.pck
/kaggle/input/dl-itba-cifar-100-2024-q-1/coarse_label_names.pck
/kaggle/input/dl-itba-cifar-100-2024-q-1/x_test.npy
/kaggle/input/dl-itba-cifar-100-2024-q-1/x_train.npy


In [85]:
from matplotlib import pyplot as plt
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [160]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Flatten, ReLU, PReLU, Activation
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras import models
from tensorflow.keras import layers, models, regularizers, constraints

In [87]:
x_train = np.load("/kaggle/input/dl-itba-cifar-100-2024-q-1/x_train.npy")
x_test = np.load("/kaggle/input/dl-itba-cifar-100-2024-q-1/x_test.npy")
y_train_coarse = np.load("/kaggle/input/dl-itba-cifar-100-2024-q-1/y_train_coarse.npy")
y_train_fine = np.load("/kaggle/input/dl-itba-cifar-100-2024-q-1/y_train_fine.npy")

In [88]:
from sklearn.model_selection import train_test_split

In [89]:
import pickle
with open("/kaggle/input/dl-itba-cifar-100-2024-q-1/fine_label_names.pck", "rb") as f:
    labels_fine = pickle.load(f)
with open("/kaggle/input/dl-itba-cifar-100-2024-q-1/coarse_label_names.pck", "rb") as f:
    labels_coarse = pickle.load(f)

In [90]:
y_train_fine.shape

(50000,)

In [91]:
x_train.shape

(50000, 32, 32, 3)

In [92]:
x_train,x_val, y_train, y_val = train_test_split(x_train, y_train_fine, test_size=0.1, stratify=y_train_fine)

In [93]:
y_val.shape

(5000,)

In [94]:
x_val.shape

(5000, 32, 32, 3)

In [95]:
x_train_horizontal_flip= x_train[:,:,::-1,:]
x_train_vertical_flip = x_train[:,::-1,:,:]
x_train_vh_flip =x_train[:, ::-1, ::-1, :]

x_train_dup = np.concatenate([x_train,x_train_horizontal_flip,x_train_vertical_flip,x_train_vh_flip],axis=0)
x_train_dup = np.concatenate([x_train,x_train_horizontal_flip],axis=0)
y_train_dup = np.concatenate([y_train, y_train,y_train, y_train], axis=0)
y_train_dup = np.concatenate([y_train, y_train], axis=0)

x_train_bis = x_train_dup.copy()
x_train_bis_bis = x_train_dup.copy()

#desplazamientos right and left 
x_train_bis[:,:,1:,:] =x_train_dup[:,:,:-1,:]
x_train_bis_bis[:,:,:-1,:] =x_train_dup[:,:,1:,:]

x_train_dup = np.concatenate([x_train_dup, x_train_bis, x_train_bis_bis],axis=0)
y_train_dup = np.concatenate([y_train_dup, y_train_dup, y_train_dup], axis=0)


In [96]:
print(f'Original dataset size: {len(x_train)}')
print(f'Augmented dataset size: {len(x_train_dup)}')

Original dataset size: 45000
Augmented dataset size: 270000


In [97]:
x_train_dup.shape, y_train_dup.shape

((270000, 32, 32, 3), (270000,))

In [123]:
from datetime import datetime
import keras

In [164]:
def create_model(input_shape=(32, 32, 3),l2_lambda=0.001):
    # Input layer for images
    inputs = layers.Input(shape=input_shape)
    
    # Flattening layer
    x = layers.Flatten()(inputs)
    
    # Normalization layer
    x = layers.Lambda(lambda x: x/np.float16(255.0))(x)
    
    # Dense layers with L1 regularization
    n = 10
    for i in range(5):
        x = Dense(2**n,
                  kernel_regularizer=regularizers.L2(l2_lambda))(x)
        x = BatchNormalization()(x)
        x = PReLU()(x)
        x = Dropout(0.2)(x)
        
    x = Dense(2**(n-3), kernel_regularizer=regularizers.L2(l2_lambda))(x)
    x = BatchNormalization()(x)
    X = Dropout(0.1)(x)
    x = layers.ReLU()(x)
    
    # Output layer with softmax for 100 classes
    fine_output = layers.Dense(100, activation='softmax', name='fine_output')(x)   
  
    # Create model
    model = models.Model(inputs=inputs, outputs=[fine_output], name=f"mlp_allnighter")
    
    return model

print(f"Success at {datetime.now().time()}")

Success at 20:44:37.830645


In [165]:
model = create_model()
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss={'fine_output': 'sparse_categorical_crossentropy'},
    metrics={'fine_output': 'accuracy' }
             )

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=8, restore_best_weights=True,verbose= 1)
tb = TensorBoard(log_dir="logs")

# Display the model summary
model.summary()

Model: "mlp_allnighter"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_27 (InputLayer)     │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_27 (Flatten)            │ (None, 3072)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda_27 (Lambda)              │ (None, 3072)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_85 (Dense)                │ (None, 1024)           │     3,146,752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_85          │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu_5 (PReLU)               │ (None, 1024)           │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_86 (Dense)                │ (None, 1024)           │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_86          │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu_6 (PReLU)               │ (None, 1024)           │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_87 (Dense)                │ (None, 1024)           │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_87          │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu_7 (PReLU)               │ (None, 1024)           │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_27 (Dropout)            │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_88 (Dense)                │ (None, 1024)           │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_88          │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu_8 (PReLU)               │ (None, 1024)           │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_28 (Dropout)            │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_89 (Dense)                │ (None, 1024)           │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_89          │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu_9 (PReLU)               │ (None, 1024)           │         1,024 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 7,515,364 (28.67 MB)

 Trainable params: 7,504,868 (28.63 MB)

 Non-trainable params: 10,496 (41.00 KB)

In [166]:
# Fit the model
history = model.fit(x=x_train_dup,   
                    y={'fine_output': y_train_dup},
                    validation_data=(x_val, {'fine_output': y_val}),
                    epochs=50,                        
                    batch_size=512,  # approximately 1000 batches per epoch                    
                    callbacks=[tb, early_stopping])

Epoch 1/50
528/528 ━━━━━━━━━━━━━━━━━━━━ 25s 25ms/step - accuracy: 0.0654 - loss: 9.9348 - val_accuracy: 0.1458 - val_loss: 8.4655
Epoch 2/50
528/528 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.1762 - loss: 7.9775 - val_accuracy: 0.1796 - val_loss: 7.0878
Epoch 3/50
528/528 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.2305 - loss: 6.5444 - val_accuracy: 0.2002 - val_loss: 6.0972
Epoch 4/50
528/528 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.2691 - loss: 5.5084 - val_accuracy: 0.2068 - val_loss: 5.4402
Epoch 5/50
528/528 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.3046 - loss: 4.7695 - val_accuracy: 0.2232 - val_loss: 4.9174
Epoch 6/50
528/528 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.3336 - loss: 4.2402 - val_accuracy: 0.2446 - val_loss: 4.4987
Epoch 7/50
528/528 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.3637 - loss: 3.8388 - val_accuracy: 0.2306 - val_loss: 4.4398
Epoch 8/50
528/528 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.3908 - loss: 3.5177 - val_accuracy: 

In [102]:
# Save the model
#model.save("accuracy0.2_manylayers_batch_norm2.keras")

In [167]:
predictions = model.predict(x_test).argmax(axis=1)
df = pd.DataFrame(predictions, columns=["Label"])

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [168]:
df

,Label
0,46
1,38
2,15
3,58
4,49
...,...
9995,70
9996,21
9997,51
9998,88


In [169]:
df.index.name = "Id"

In [170]:
df.to_csv("submission.csv")